# **9주차: Attention & Transformer**

In [1]:
# install package
!pip install torch, torchvision, torchtext

ERROR: Invalid requirement: 'torch,': Expected end or semicolon (after name and no valid version specifier)
    torch,
         ^


## **어텐션(Attention)**

### **시퀀스투시퀀스(Seq2Seq)**
seq2seq 모델은 두 개의 RNN으로 구성된 모델이다.

인코더(encoder)는 입력 시퀀스(문장)를 읽고 컨텍스트 벡터라는 문장에 대한 정보가 응축된 벡터를 출력한다.

디코터(decoder)는 컨텍스트 벡터로부터 출력 시퀀스를 생성한다.

<video controls width="640" height="360">
  <source src="https://jalammar.github.io/images/seq2seq_5.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
해당 영상에서 볼 수 있 듯 seq2seq 모델은 단일 RNN과 달리 전체 input을 살펴본 후 고정된 사이즈의 context vector를 출력해 전체적인 맥락 파악이 가능하다.
 </br> </br>

그러나 아래와 같은 한계가 존재한다.

#### **1. 기울기 소실(gradient vanishing)**

input sequence의 길이가 길어질수록 즉, 먼 거리의 단어일수록 상호작용이 어렵다.


<img src="https://velog.velcdn.com/images/jody1188/post/4d1d2b04-014c-4039-ad3b-44b50912884e/image.png" width="500">


예를 들어 위 문장에서 chef의 동사는 was이지만 관계대명서 who로 인해 주어 동사의 상호작용을 하기 위해선 sequence length만큼의 연산이 이루어져야하지만 기울기 소실 (gradient vanishing) 문제로 서로 상호작용하기 어렵다.
     </br> </br>

#### **2. 정보 손실**

하나의 고정된 크기의 벡터인 context vector에 모든 정보를 압축해 정보 손실의 한계가 존재한다.

이외에도 병렬 연산 불가능, 입력 문장이 길면 효율적인 학습을 못하는 문제가 있다.

### **attention 메카니즘 (query, key, value)**

어텐션 매커니즘은 입력 문장이 긴 상황에서 효율적인 학습을 위해 중요한 단어에 집중하여 decoder에 바로 전달하는 방법이다.

<video controls width="640" height="360">
  <source src="https://jalammar.github.io/images/seq2seq_6.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

영상 출처 : https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/

어텐션의 기본 아이디어는 디코더에서 출력 단어를 예측하는 매 시점(step)마다 인코더의 입력 시퀀스를 다시 참고하는 것이다.

![image.png](attachment:image.png)

#### **직관적인 흐름**

**1. 인코더의 hidden state(h1, h2, h3, h4)들을 step별로 구한다. (그림에선 파란색 박스를 나타낸다. )**

**2. 각각 step의 hidden state(h1, h2, h3, h4)에 이전 step 디코더의 hidden state(빨간색 박스)인 $S_{i-1}$를 각각 dot-product하여 점수를 부여한다. 이때 이 점수는 attention score이다.**

$유사도 점수(Attention Score) = s_{i-1} \cdot h_j$

예를 들어 그림에서 디코더는 출력 문장의 첫 단어("Je")를 생성하려고 한다. 이때 이전 디코더의 hidden state인 $s_0$를 사용하고 $s_0$는 디코더의 이전 hidden state가 없으니 인코더에서 마지막으로 생성된 hidden state로 설정한다.

**3. 계산된 유사도 점수는 소프트맥스 함수(softmax)로 정규화한다. (점수 합이 1)**

해당 그램에서 attention weights 부분이 유사도 점수를 정규화한 값이다. 해당 값들은 출력 단어("Je")를 생성하는 데 얼마나 중요한 지 나타낸다.

**4. context vector를 생성한다.**

각 인코더 hidden state ($h_1, h_2, h_3, h_4$)에 정규화한 유사도 점수를 각각 곱해준다.

$0.5×h_1, 0.3×h_2, 0.1×h_3, 0.1×h_4$

이 가중치를 합산하여 하나의 벡터로 만드는 데 이 벡터가 context vector이다.

즉, 이 contect vector는 "Je"를 생성하는 데 필요한 입력 문장의 정보를 담고 있다.

**5. 디코더가 출력을 생성한다.**

Context vector와 이전 hidden state($s_{i-1}$)인 $s_0$를 사용하여 출력 단어 "Je"를 생성한다.

같은 과정을 반복하여 그림과 같이 "suis", "étudiant", "</s>"와 같은 다음 단어들도 생성한다.

#### **어텐션 함수(Attention Function)**

<img src="https://velog.velcdn.com/images%2Fsjinu%2Fpost%2F0c796b58-065e-4b45-bbd7-712d3fc1d05b%2Fimage.png" width="500">

$$
Attention(Q,K,V)=Attentionvalue \\

Q:Query \\
K:Key \\
V:Value \\
$$

**1. Query : 현재 디코터가 어떤 출력 단어를 생성해야하는 지 나타낸다.(구글 검색 시 검색 상자에 입력하는 텍스트)**

디코더의 이전 hidden state인 $s_{i-1}$이 쿼리 역할을 한다.

**2. Key : 입력 문장의 각 단어가 가지고 있는 정보를 나타낸다. (기사 제목으로 표시되는 결과)**

인코더에서 생성된 각 hidden state ($h_1, h_2, h_3, h_4$)가 키 역할을 한다.

**3. Value : 입력 문장의 실제 정보이다. (그 안의 내용)**

인코더의 각 hidden state ($h_1, h_2, h_3, h_4$)가 밸류 역할을 한다.

여기서 Key는 단순히 **찾는 기준**(단어의 위치 정보)이고 Value는 **활용하는 정보**(단어의 의미 정보)이다.


**Query와 Key를 비교**하여 유사도 점수(attention score)를 계산하고 각 Value와 가중치를 결합하여 Context Vector를 만든다.

#### **어텐션 종류**

유사도 점수 (attention score)를 구하는 방법은 여러가지이며 이에 따라 다양한 종류의 어텐션이 있다.

#### **Scaled Dot-Product Attention**

$Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{D_K}}) V \in R^{N\times D_v} \\ Queries : Q \in R^{N\times D_k},q_i\in R^{1\times D_k} \\ Keys : K \in R ^{N \times D_k} , k_i \in R^{1\times D_k} \\ Values : V \in R^{N \times D_v} , v_i \in R^{1 \times D_v}$

Query, Key, Value 간의 유사도를 바탕으로 데이터를 가중 평균하여 각 Query에 대한 중요한 정보를 추출한다.

#### **Multi-Head Attention**

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fcx6fS9%2FbtrTFmhGL1b%2FliEpkqd2KPl2qwvXHPPYD0%2Fimg.png" width="500">

멀티 헤드 어텐션은 어텐션의 확장 형태로 여러개의 어텐션 헤드를 병렬로 실행하여 데이터를 학습한다.

$\begin{matrix} MultiHead(Q,K,V) &=& \textbf{Concat}(head_1,...,head_h)W^O, \ head_i &=& \textbf{Attention}(QW_i^Q,KW_i^K,VW_i^V) \end{matrix}$

각 $head_i$는 독립적인 Scaled Dot-Product Attention을 계산한다.

얘를 들어 한 헤드는 문장의 문맥적 의미를 학습하고 다른 헤드는 단어 간의 구문적 관계를 학습할 수 있다.

Concat은 $h$개의 어텐션 헤드 출력을 연결하여 하나의 큰 벡터로 병합한다.


이외에도 다양한 어텐션 방법들이 존재한다.

<img src="https://velog.velcdn.com/images%2Fsjinu%2Fpost%2F4cf43124-67c3-4160-9478-a46567bfc5d2%2Fimage.png" width="500">

## **트랜스포머(Transformer)**


#### **self-attention 메카니즘**

self-attention은 입력 시퀀스 내의 각 요소가 다른 요소들과의 관계를 학습하도록 설계된 매커니즘이다.

<img src="https://velog.velcdn.com/images/nanhungrybin/post/c4a92b2c-38de-4919-88f3-7c2e3e17cce4/image.png" width="500">

입력 데이터의 각 요소가 자기 자신을 포함한 모든 요소와 상호작용한다.

입력 요소 간의 관계를 계산하여 중요도를 나타내는 가중치를 생성한다(어텐션 가중치). 계산된 가중치를 사용하여 중요도가 반영된 입력 요소 들의 가중합을 생성한다.

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)V
$$

**순열 등변성(Permutation Equivariance)**

Self-Attention 매커니즘은 순열 등변성 성질을 갖고 있는데 입력 벡터의 순서를 바꾸더라도 그에 맞게 동일한 연산을 적용하는 성질이다.

이 특성으로 인해 순서 정보가 부족한 경우 이를 보완하기 위해 포지셔널 인코딩(Positional Encoding)이 사용된다.

**positional vector**

위치 정보를 포함하고 있는 새로운 query, key, value를 만드는데 필요한 position vector는 아래와 같이 정의한다.

$p_i ∈ R^d, for i∈ {1,2,...,T}$

이렇게 정의한 position vector는 각 query, key, value에 더해서 위치 정보를 부여한다.

짝수 인덱스 : $
PE(\text{pos}, 2i) = \sin\left(\text{pos} \cdot \frac{1}{10000^{\frac{2i}{d_{\text{model}}}}}\right)$

홀수 인덱스 : $
PE(\text{pos}, 2i+1) = \cos\left(\text{pos} \cdot \frac{1}{10000^{\frac{2i}{d_{\text{model}}}}}\right)$


**피드 포워드 신경망 (feed-forward network)**

self-attention은 단순 가중합 형태의 선형결합이기 때문에 feed-foward network를 추가한다. 이때 피드 포워드 신경망은 ReLu함수를 추가하여 비선형으로 만들어 각 layer들은 다른 파라미터를 갖는다.

**masking**

이후 병렬 계산을 하여 미래의 시퀀스 단어를 연산에 이용하지 못하게 마스크처리를 한다.

<img src="https://velog.velcdn.com/images/nanhungrybin/post/a6340742-e067-42dc-bcf5-55b572ea9f2f/image.png" width="500">

cf. cross attention : cross-attention은 입력과 출력 간의 순서를 보존하면서 관계를 학습한다(seq2seq model의 attention과 유사). 이 경우 순열 등변성의 성질은 없다.

#### **트랜스포머 모델**

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FHJodK%2FbtsHDG0lvfS%2FPb4Bpp3soKDBdWrx5b0oTK%2Fimg.webp" width="500">

**1. 트랜스포머 인코더**

input값으로 워드 임베딩이 들어오면 position encoding 값과 더해져 멀티 헤드 어텐션 레이어로 들어간다. 셀프 어텐션을 거쳐 나온 output에 residual connection을 수행한다.

이는 비선형함수를 통해 나온 output에 input을 더해주는 방식으로 gradient를 보장해 안정적인 학습을 보장한다.

LayerNorm은 layer normalization으로 모든 feature에 대해 평균과 분산을 구하여 각 input을 정규화한다. 이 역시 gradient를 안정화시킨다.

이후 피드 포워드에 들어가게 된다.

**2. 트램스포머 디코더**

마지막 인코더 블록 종료 후 디코더 과정으로 넘어간다.

처음 워드 임베딩과 포지셔널 인코딩값을 더해 마스크를 넣은 멀티헤드 어텐션에 들어간다. 디코더는 해당 step의 다음 시점을 참고하면 안되므로 making을 사용한다. 이후 layer normalization을 거치고 multi-head cross attention에 들어간다.

여기서 cross attention을 사용하는 이유는 입력 문장의 관계를 학습하여 인코더 출력을 참조하도록 돕기 위해 순서를 보존하는 cross attention을 사용한다.

피드 포워드 신경망을 거쳐 잔차 연결 및 정규화를 한 후 최종 출력을 한다.

#### **RNN 계열 모델 특징 및 장단점**

**RNN 계열 모델로는 LSTM, GRU 등이 포함된다.**

특징

1. 순차적 처리 : 입력 데이터를 시간 순서대로 처리한다
2. 메모리 효율 : 이전 상태를 재사용하여 처리한다
3. 문맥 학습 : 과거 데이터를 기반으로 현재 데이터를 학습한다

장점

1. 시간 순서를 반영해 시간 의존성이 강한 데이터(ex. 시계열, 음성 데이터 등)에 적합하다
2. 구조가 직관적이고 이해하기 쉽다.

단점

1. 데이터가 순차적으로 처리되어 병렬화가 어렵다
2. 장기 의존성 문제 : 긴 시퀀스에서 과거 정보를 학습하는데 어려움이 있다
3. 기울기 소실 문제 : 순환 구조로 기울기 소실/폭발 가능성이 있다

**트랜스포머 모델**

특징

1. 입력 데이터를 동시에 처리 가능해 병렬화가 가능하다
2. 데이터 학습 시 중요한 부분을 선택적으로 학습할 수 있다

장점

1. 장기 의존성 학습 : 긴 문맥에서 관계를 효율적으로 학습한다
2. 모델 크기를 쉽게 확장 가능하다
3. NLP, 이미지 처리, 음성 처리 등 다양한 분야에 사용가능하다

단점

1. 메모리 사용량 : 모든 입력 간의 관계를 계산해야해 메모리 사용량이 높다
2. 복잡성 : 구조가 복잡하여 구현과 하이퍼파라미터 튜닝이 어려울 수 있다

## **비전 트랜스포머 (Vision Transformer (ViT))**

대규모 데이터셋에 강한 비전 트랜스포머(ViT)는 트랜스포머 모델을 기반으로 이미지 처리에 적용된 딥러닝 아키텍처로 CV에 접목시켰다.


#### **각 layer 역할 및 특징**

<img src="https://velog.velcdn.com/images/pre_f_86/post/d012be1c-ece1-40e3-9402-a0951fcc955a/image.png" width=400> </br> </br>

**1. 패치 임베딩(patch embedding)**

이미지는 2D이므로 1차원으로 변환하기 위한 작업인 패치 임베딩을 수행한다.

<img src="https://velog.velcdn.com/images/pre_f_86/post/55e4ddaa-2969-4436-8c36-633a96156f2f/image.png" width="300">

위 그림은 4×4×2의 이미지를 2×2크기의 패치로 나누어 flatten 작업을 해준 후 4크기의 벡터로 임베딩한다.

**2. 클래스 토큰(class token)**

이미지 분류를 위한 클래스 토큰을 사용한다.

input embedding 앞에 [Class]Patch(임의의 랜덤한 값)를 넣어주고 패치는 이후 트랜스포머 인코더의 출력 중 맨 앞에 대응되어 MLP Head에 입력으로 들어가 classification 작업에 사용된다.

최종적으로 패치 순서 정보를 전달하기 위한 포지셔널 인코딩(Positional Encoding)을 하여 위치에 대한 정보를 추가한다.

해당 과정들은 아래의 그림과 같이 표현할 수 있다.

<img src="https://velog.velcdn.com/images/pre_f_86/post/eee5703a-2b54-4e3f-8d64-3307dcd88f3b/image.png" width="500">

최종적인 임베딩 벡터들은 트램스포머와 동일하게 멀티 헤드 어텐션과 레이어 정규화를 진행한 후 최종적으로 MLP head를 지나 classification output이 나온다.

**전체적인 흐름**

<video controls width="640" height="360">
    <source src="https://mblogvideo-phinf.pstatic.net/MjAyMzAxMTRfMzAw/MDAxNjczNjkyODQzMjcz.IaJlWbmiPcuCFsPqaryg3l29ClV0Cqftg6CqMYHlBJsg.p4ZQLBvm4e8OWZbC0pSovPno54_5buISSwvQComDd9Ig.GIF.nueyet/img.gif?type=mp4w800" type="video/mp4">
</video>

#### **비전 트랜스포머 의의**

1. 이미지 테스크를 트랜스포머구조 모델에서 구현하였다
2. 비전 테스크에서 CNN을 사용하지 않고 트랜스포머의 멀티 헤드 어텐션 구조를 사용하여 충분한 퍼포먼스를 구현하였다
3. 학습 과정에서의 계산 자원은 훨씬 적게 소모하였다

#### **한계**

데이터셋의 양이 충분하지 않은 경우 기존 CNN 대비 성능이 낮다. 이는 CNN에 비해 유도 편향(Inductive Bias)가 부족하여 많은 양의 데이터가 없이 일반화가 제대로 이루어지지 않은 이유이다.

큰 사이즈의 데이터셋의 경우 CNN보다 더 높은 성능을 나타낸다.

cf. 유도 편향(Inductive Bias) : 학습에 사용되지 않은 데이터에 대해서 어떤 것에 대해 예측할 때 정확한 예측을 위해 사용하는 추가적인 가정

CNN은 필터가 이미지의 모든 영역을 스캔하는 방식이지만 ViT는 이미지를 패치로 나누어 작동하며 MLP는 한 패치 내부에서만 Fully Connected 형식으로 작동하기 때문에 Translation Equivariance(입력 위치가 달라져도 출력은 동일) 특성이 사라지게 된다.

<img src="https://velog.velcdn.com/images/pre_f_86/post/71a5c5c9-b1c5-4ebe-a06e-9ef5116116bc/image.png" width="200">

따라서 ViT가 CNN에 비해 유도 편향이 더 적다